In [1]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import plotly.graph_objects as go
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
df = yf.download(ticker, period="1y", interval="1h")


In [ ]:
df.columns

In [2]:
# For other spreadsheet use this in reading csv
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)


In [3]:
df = df.iloc[:500]

In [13]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)])

fig.update_layout(
    xaxis=dict(type='category', nticks=20),
    xaxis_rangeslider_visible = False
)

fig.show()


In [ ]:
df.head()

In [4]:
conditions = [
    df.Close > df.Open,
    df.Close < df.Open
]

choices = ['Bull', 'Bear']

df['Candle'] = np.select(conditions, choices, default='Doji')
df['Swing_Low'] = None
df['Swing_High'] = None

In [5]:
hh = float('-inf')
hl = float('-inf')

lh = float('inf')
ll = float('inf')

current_open = df.Open.iloc[-1]
current_high = df.High.iloc[-1]
current_low = df.Low.iloc[-1]
current_close = df.Close.iloc[-1]

previous_open = df.Open.iloc[-2]
previous_high = df.High.iloc[-2]
previous_low = df.Low.iloc[-2]
previous_close = df.Close.iloc[-2]

In [ ]:
df.head(50)

In [6]:
df['Swing_High'] = None
df['Swing_Low'] = None
df['Trend'] = None

hh = float('-inf')
pullback_count = 0
swing_high = None
previous_high = float('-inf')
previous_low = float('inf')

for i, row in df.iterrows():
    current_high = row['High']
    current_low = row['Low']
    current_close = row['Close']
    candle_type = row['Candle']
    
    # Logic to update hh and pullback count
    if current_high > previous_high:
        hh = max(hh, current_high)
        swing_high = hh 
        pullback_count = 0
    elif candle_type == 'Bear' and current_high < swing_high:
        pullback_count += 1
        
    # Check for pullback and update Swing_High
    if pullback_count >= 2 and swing_high is not None:
        df.at[i, 'Swing_High'] = swing_high
        if current_low < previous_low:
            swing_low = current_low 
            df.at[i, 'Swing_Low'] = swing_low
    
    # Check for trend and update
    if current_close > swing_high and swing_high is not None:
        df.at[i, 'Trend'] = 'Uptrend'
        hh = current_high
        pullback_count = 0  # Reset pullback count after confirming trend
    
    # Update previous values for next iteration
    previous_high = current_high
    previous_low = current_low

In [12]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)])



swing_highs = df[df['Swing_High'].notnull()]
fig.add_trace(go.Scatter(x=swing_highs.index, y=swing_highs['Swing_High'], mode='markers', name='Swing Highs'))
swing_lows = df[df['Swing_Low'].notnull()]
fig.add_trace(go.Scatter(x=swing_lows.index, y=swing_lows['Swing_Low'], mode='markers', name='Swing Lows'))


fig.update_layout(
    height=600,
    xaxis=dict(type='category', nticks=20),
    xaxis_rangeslider_visible = False
)


fig.show()

In [ ]:
df